In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json

client_id = ''
client_secret = ''

def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s]Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e :
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=\"%s\"&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    url = base + node + parameters
    responseDecode = getRequestUrl(url)     

    if(responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S %z')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt':cnt, 'title':title, 'description':description,
                       'link':org_link, 'pDate':pDate})
    return

def main():
    node = 'news' 
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []
    
    jsonResponse = getNaverSearch(node, srcText, 1, 100)
    total = jsonResponse['total']
    
    while((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt+= 1
            getPostData(post, jsonResult, cnt)  
            
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)  
        
    print('전체 검색 : %d 건' %total)
        
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys = True, ensure_ascii = False)
            
        outfile.write(jsonFile)
            
    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':
    main()                  

[2024-09-19 11:19:15.585048]Url Request Success
[2024-09-19 11:19:15.736950]Url Request Success
[2024-09-19 11:19:15.903423]Url Request Success
[2024-09-19 11:19:16.099479]Url Request Success
[2024-09-19 11:19:16.279449]Url Request Success
[2024-09-19 11:19:16.464556]Url Request Success
[2024-09-19 11:19:16.672947]Url Request Success
[2024-09-19 11:19:16.865463]Url Request Success
[2024-09-19 11:19:17.104303]Url Request Success
[2024-09-19 11:19:17.314099]Url Request Success
HTTP Error 400: Bad Request
[2024-09-19 11:19:17.391865] Error for URL : https://openapi.naver.com/v1/search/news.json?query="STO"&start=1001&display=100
전체 검색 : 22053 건
가져온 데이터 : 1000 건
STO_naver_news.json SAVED


국토 교통부
https://www.molit.go.kr/USR/NEWS/m_71/dtl.jsp?lcmspage=2&id=95090188
https://www.molit.go.kr/USR/NEWS/m_71/dtl.jsp?lcmspage=2&id=95090184
https://www.molit.go.kr/USR/NEWS/m_71/dtl.jsp?lcmspage=2&id=95090173


기획재정부
https://www.moef.go.kr/nw/nes/detailNesDtaView.do?searchBbsId1=MOSFBBS_000000000028&searchNttId1=MOSF_000000000070670&menuNo=4010100
https://www.moef.go.kr/nw/nes/detailNesDtaView.do?searchBbsId1=MOSFBBS_000000000028&searchNttId1=MOSF_000000000070655&menuNo=4010100
https://www.moef.go.kr/nw/nes/detailNesDtaView.do?searchBbsId1=MOSFBBS_000000000028&searchNttId1=MOSF_000000000070650&menuNo=4010100

